In [1]:
#Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## DnaA boxes

DnaA protein are binded to regions called dnaA boxes in the region knowed as ori, where the replication processes begin. Finding ori is one of the key tasks to understand how cells replicate. First lets search for frequent characters in ori, because some nucleotide string appear surprisingly often in small regions of genome. 
We will start with a bacterium called Vibrio cholerae, and then design a computational approach for finding ori in other bacteria genomes.

Here is the nucleotide sequence appearing in the ori of Vibrio cholerae:

![ORI vibrio](data/Screenshot_20200712_003201.png)

## K-mer

One possible approach is design a "sliding window" that will go through the text checking where each substring of the input text matches with the pattern that we are looking for

K-mer pseudocode:

![Pseudocode k-mer](data/k-mer.png)

Using the python language, let's implement the pseudocode.

ps: Obviously, in python language, there is a lot of ways of doing so, many of them much more efficient and simple, but lets keep the code more like the pseudocode

In [2]:
def PatternCount(text, pattern):
	count = 0

	for i in range(len(text) - len(pattern)):
		if text[i: i + len(pattern)] == pattern:
			count += 1
	return count

In [3]:
#Example
text = open("data/ori_vibrio.txt", "r")
texto = text.read()
count = PatternCount(texto, 'CCG')
print(count)

13125


## Frequent word
We say that a pattern is a most frequent k-mer in the input text if it maximizes Count(Text, Pattern) among all k-mers. For instance, ACTAT is a most frequent 5-mer of ACAACTATGCATACTATCGGGAACTATCCT.

One algorithm for finding the most frequent k-mers in a string checks all k-mers appearing in this input string, then computes how many times each k-mer appears in the string. To implement this FrequentWords algorithm, lets make an array Count, where Count(i) stores Count(Text, Pattern) for Pattern = Text(i, k).

Frequentwords pseudocode:
![Pseudocode frequentwords](data/frequentwords.png)

In [4]:
def FrequentWordsProblem(text, k):
	frequentPatterns = []
	count = np.zeros(shape=(len(text) - k + 1))

	for i in range(len(text) - k):
		pattern = text[i: i + k]
		count[i] = PatternCount(text, pattern)

	max_count_indicies = np.where(count == np.max(count))
	
	for i in max_count_indicies[0]:
		if text[i:i+k] not in frequentPatterns:
			frequentPatterns.append(text[i:i+k])
	return frequentPatterns

In [5]:
frequent = FrequentWordsProblem('ACGTTGCATGTCGCATGATGCATGAGAGCT', 4)
print(frequent)

['GCAT', 'CATG']


FrequentWords finds most frequent k-mers, but is not very efficient. Each call to PatternCount function checks whether the k-mer pattern appears in all positions of the text. Since each k-mer requires |Text| − k + 1 such checks, each one requiring as many as k comparisons, the overall number of steps of PatternCount is (|Text| − k + 1) · k. Furthermore, FrequentWords must call PatternCount |Text| − k + 1 times (once for each k-mer of Text), so that its overall number of steps is (|Text| − k + 1) · (|Text| − k + 1) · k.

## Searching for hidden messages in multiple genomes

Not all bacteria has the same DnaA boxes. Let's take a look at some the ori region of Thermotoga petrophila and Vibrio cholerae.

In [6]:
#Thermotoga petrophila 
frequent = FrequentWordsProblem('aactctatacctcctttttgtcgaatttgtgtgatttatagagaaaatcttattaactga'+ \
    'aactaaaatggtaggtttggtggtaggttttgtgtacattttgtagtatctgatttttaa' + \
    'ttacataccgtatattgtattaaattgacgaacaattgcatggaattgaatatatgcaaa' + \
    'acaaacctaccaccaaactctgtattgaccattttaggacaacttcagggtggtaggttt' + \
    'ctgaagctctcatcaatagactattttagtctttacaaacaatattaccgttcagattca' + \
    'agattctacaacgctgttttaatgggcgttgcagaaaacttaccacctaaaatccagtat' + \
    'ccaagccgatttcagagaaacctaccacttacctaccacttacctaccacccgggtggta' + \
    'agttgcagacattattaaaaacctcatcagaagcttgttcaaaaatttcaatactcgaaa' + \
    'cctaccacctgcgtcccctattatttactactactaataatagcagtataattgatctga', 9)

print(frequent)

['acctaccac']


In [7]:
#Vibrio cholerae
frequent = FrequentWordsProblem('atcaatgatcaacgtaagcttctaagcatgatcaaggtgctcacacagtttatccacaac'+ \
'ctgagtggatgacatcaagataggtcgttgtatctccttcctctcgtactctcatgacca'+ \
'cggaaagatgatcaagagaggatgatttcttggccatatcgcaatgaatacttgtgactt'+ \
'gtgcttccaattgacatcttcagcgccatattgcgctggccaaggtgacggagcgggatt'+ \
'acgaaagcatgatcatggctgttgttctgtttatcttgttttgactgagacttgttagga'+ \
'tagacggtttttcatcactgactagccaaagccttactctgcctgacatcgaccgtaaat'+ \
'tgataatgaatttacatgcttccgcgacgatttacctcttgatcatcgatccgattgaag'+ \
'atcttcaattgttaattctcttgcctcgactcatagccatgatgagctcttgatcatgtt'+ \
'tccttaaccctctattttttacggaagaatgatcaagctgctgctcttgatcatcgtttc', 9)

print(frequent)

['atgatcaag', 'ctcttgatc', 'tcttgatca', 'cttgatcat']


## The Clump Finding Problem

We can define a k-mer as a "clump" if it appears t times within a interval L of the genome. So instead of finding clumps of a specific k-mer, let’s try to find every k-mer that forms a clump in the genome, because different genomes may use a completely different hidden messages.
We can solve the Clump Finding Problem by applying our FrequentWords algorithm to each window of length L in Genome. If your FrequentWords implementation is not very efficient, then such an approach may be impractical. Recall that FrequentWords has O(L2 · k) running time. Applying this algorithm to each window of length L in Genome will result in an algorithm with O(L2 · k · |Genome|) running time. 

Firstly, let's solve the clump finding problem with our implementation of FrequentWords without frequency array. Then, let's improve our solution to FrequentWords algorithm, solve the clump finding 
problem and see the performances for both of the approaches.


Pseudocode:
![Pseudocode clump](data/clump.png)


Let's do some modifications in our FrequentWords function to check which k-mers appears at least T or more times, instead of only get the maximum value of the ocurrencies of the k-mer. Then, let's implement the above pseudocode for the clumping finding algorithm.

In [25]:
def FrequentWordsProblem(text, k, t):
	frequentPatterns = []
	count = np.zeros(shape=(len(text) - k + 1))


	for i in range(len(text) - k):
		pattern = text[i: i + k]
		count[i] = PatternCount(text, pattern)

	if t == None:
		t = np.max(count) 

	max_count_indicies = np.where(count >= t)
	
	for i in max_count_indicies[0]:
		if text[i:i+k] not in frequentPatterns:
			frequentPatterns.append(text[i:i+k])
	return frequentPatterns

In [26]:
def ClumpingFinding(text, k, l, t):
	count_kmer = []
	count = []
	for i in range(len(text) - l):
			k_mer = text[i:i+l]
			freq = FrequentWordsProblem(k_mer, k, t)
			count = [x for x in freq if x not in count_kmer]
			if len(count) != 0: count_kmer.extend(count)

	return count_kmer

In [15]:
text = open("data/clump_test.txt", "r")
texto = text.read()
%timeit print(ClumpingFinding(texto, 11, 566, 18))

['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
2min 18s ± 270 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Using the %timeit command, our implementation of clumping Finding will execute a couple of times, which allow us get the mean from the execution times to evaluate our performances. The algorithm returned 'AAACCAGGTGG' as answer for our test.

Let's improve our FrequentWords to use an array of frequences.

We will first order all 4k k-mers lexicographically and then convert them into the 4k different integers between 0 and 4k − 1. Given an integer k, we define the frequency array of a string Text as an array of length 4k, where the i-th element of the array holds the number of times that the i-th k-mer (in the lexicographic order) appears in Text.

![fast_freq](data/fast_freq2.png)

The pseudocode below generates a frequency array by first initializing every element in the frequency array to zero (4k operations) and then making a single pass down Text (approximately |Text| · k operations). For each k-mer Pattern that we encounter, we add 1 to the value of the frequency array corresponding to Pattern. As before, we refer to the k-mer beginning at position i of Text as Text(i, k).

![fast_freq_pseudo](data/computing_freq.png)


Next, we can find all most frequent k-mers by finding all k-mers corresponding to the maximum elements in frequency array.

![fast_freq_pseudo2](data/fast_freq.png)


In [16]:
def ComputingFrequencies(text, k):
	frequencyArray = np.zeros(4**k)
	for i in range(len(text) - k + 1):
		pattern = text[i: i+k]
		j = PatternToNumber(pattern)
		frequencyArray[j] += 1
	return frequencyArray

In [30]:
def FasterFrequentWords(text, k, t):
	frequentPatterns = []
	frequencyArray = ComputingFrequencies(text, k)
	
	if t == None:
		t = np.max(frequencyArray)

	max_count_indicies = np.where(frequencyArray >= t)
	
	for i in max_count_indicies[0]:
		pattern = NumberToPattern(i,k)
		if pattern not in frequentPatterns:
			frequentPatterns.append(pattern)
	return frequentPatterns

The pseudocode below slides a window of length L down Genome. After computing the frequency array for the current window, it identifies (L, t)-clumps simply by finding which k-mers occur at least t times within the window. To keep track of these clumps, our algorithm uses an array Clump of length 4k whose values are all initialized to zero. For each value of i between 0 and 4k − 1, we will set Clump(i) equal to 1 if NumberToPattern(i, k) forms an (L, t)-clump in Genome.

![clump_pseudo2](data/clump2.png)

Before implement the clumpFinding, let's implement some auxiliary functions.
We will need a function to transform a pattern in some number. If we remove the final symbol from all lexicographically ordered k-mers, the resulting list is still ordered lexicographically. In the case of DNA strings, every (k − 1)-mer in the resulting list is repeated four times.

![pattern2number](data/pattern2number.png)

Pseudocode:

![pattern2numberpseudo](data/patter2number2.png)


In [20]:
def PatternToNumber(pattern):
	if len(pattern) == 0:
		return 0
	LastSymbol = {'A':0, 'C': 1, 'G': 2, 'T': 3}
	symbol = LastSymbol[pattern[-1]]
	Prefix = pattern[:-1]
	return 4 * PatternToNumber(Prefix) + symbol

In [21]:
PatternToNumber('AGT')

11

To make the inverse function, NumberToPattern, basically, we need to follow this equation:

PatternToNumber(Pattern) = 4 · PatternToNumber(Prefix(Pattern)) + SymbolToNumber(LastSymbol(Pattern))

Let's see an example after computed NumberToPattern(9904, 7)

![number2pattern](data/number2pattern.png)

Pseudocode:

![number2pattern](data/number2patternps.png)



In [22]:
def NumberToPattern(index, k):
	LastSymbol = {0:'A', 1: 'C', 2: 'G', 3: 'T'}
	
	if k == 1:
		return LastSymbol[index]

	prefixIndex = index // 4
	r = index % 4
	symbol = LastSymbol[r]
	PrefixPattern = NumberToPattern(prefixIndex, k - 1)
	return PrefixPattern + symbol

In [23]:
NumberToPattern(45, 4)

'AGTC'

Now, let's proceed and implement the Clumping Finding pseudocode that we saw before. Again, using the %timeit command, we will execute our function a couple of times and compare the results with our previous implementation using the same file

In [31]:
def ClumpingFinding2(text, k, l, t):
	count_kmer = []
	count = []
	for i in range(len(text) - l):
			k_mer = text[i:i+l]
			freq = FasterFrequentWords(k_mer, k, t)
			count = [x for x in freq if x not in count_kmer]
			if len(count) != 0: count_kmer.extend(count)

	return count_kmer

In [32]:
text = open("data/clump_test.txt", "r")
texto = text.read()
%timeit print(ClumpingFinding2(texto, 11, 566, 18))

['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
['AAACCAGGTGG']
24.5 s ± 45.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


To improve ClumpFinding, we observe that when we slide our window of length L one symbol to the right, the frequency array does not change much, and so regenerating the frequency array from scratch is inefficient. For example, the figure below shows the frequency arrays (k = 2) for the 13-mers Text = AAGCAAAGGTGGG and Text′ = AGCAAAGGTGGGC starting at positions 0 and 1 of the 14-mer AAGCAAAGGTGGGC. These two frequency arrays differ in only two elements corresponding to the first k-mer in Text (AA) and the last k-mer in Text’ (GC). 

![improveClumps](data/number2patternps.png)